In [2]:
import os
import json
from langchain.chat_models import ChatOpenAI 
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredFileLoader 
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.globals import set_llm_cache, set_debug
from langchain.storage import LocalFileStore
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.utils.math import cosine_similarity

In [3]:
# 언어 모델 초기화
print("🔧 Initializing language model...")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 캐시 디렉토리 설정
print("💾 Setting up cache directory...")
cache_dir = LocalFileStore("./.cache/embeddings/course_computer_networks")

# 텍스트 스플리터 설정
print("✂️ Setting up text splitter...")
spliter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# 문서 로드
print("📄 Loading document...")
loader = UnstructuredFileLoader("./dummy.csv")

# 문서 분할
print("🔍 Splitting document into chunks...")
docs = loader.load_and_split(text_splitter=spliter)

# 임베딩 모델 설정
print("🧠 Initializing embeddings model...")
embeddings = OpenAIEmbeddings()

# 캐싱 임베딩 설정
print("🗂️ Setting up cached embeddings...")
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

# 벡터스토어 생성
print("📊 Creating vector store from documents...")
vectorstore = FAISS.from_documents(docs, cache_embeddings)

# 검색기 설정
print("🔎 Setting up retriever...")
retriever = vectorstore.as_retriever()

print("✅ Process completed!")

🔧 Initializing language model...


/Users/seochan/ai-server/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


💾 Setting up cache directory...
✂️ Setting up text splitter...
📄 Loading document...
🔍 Splitting document into chunks...


/Users/seochan/ai-server/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 0.4.0. An updated version of the class exists in the langchain-unstructured package and should be used instead. To use it run `pip install -U langchain-unstructured` and import as `from langchain_unstructured import UnstructuredLoader`.
  warn_deprecated(


🧠 Initializing embeddings model...
🗂️ Setting up cached embeddings...
📊 Creating vector store from documents...
🔎 Setting up retriever...
✅ Process completed!


/Users/seochan/ai-server/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
# 프롬프트 템플릿 생성
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            """
            당신은 AI 어시스턴트입니다. 
            사용자가 질문에 대한 답변을 찾을 수 있도록 돕습니다.
            
            사용자 질문과 유사한 질의응답 선 사례 : {context}

            위 선사례에 기반하여 사용자의 질문에 대한 답변을 친절하게 설명해주세요.
            """
        ),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)


In [8]:
def rag_pipeline(question):
    # 입력된 질문에 대한 임베딩 생성
    print("🧠 Embedding the input question...")
    question_embedding = embeddings.embed_documents([question])[0]    
    # 기존 질문과 비교
    print("🔍 Comparing with existing documents...")
    results = retriever.get_relevant_documents(question)
    similarities = {}
    
    for doc in results:
        doc_content = doc.page_content.strip()
        if not doc_content:
            continue
        
        doc_embedding = embeddings.embed_documents([doc_content])[0]
        similarity = cosine_similarity([question_embedding], [doc_embedding])[0][0]
        similarity_percentage = similarity * 100
        similarities[doc_content] = similarity_percentage

    # 가장 유사한 질문 찾기
    most_similar_question = max(similarities, key=similarities.get, default=None)
    if most_similar_question:
        max_similarity = similarities[most_similar_question]
        

        if max_similarity > 80.0:  # 유사도가 충분히 높은 경우 관련 답변 사용
            context = f"질문이 '{most_similar_question}'와 매우 유사합니다."
            print(f"\n가장 유사한 질문: '{most_similar_question}'")
            print(f"유사도: {max_similarity:.2f}%")
            prompt = prompt_template.format(context=context, question=question)
            output = llm.invoke(prompt)
            return { "LLMAnswer": output.content, "userQuestion": question}
    
    return {"LLMAnswer": None, "userQuestion": question}

In [10]:
# 메인 함수
def main(user_input):
    response = rag_pipeline(user_input)
    print("Assistant:", response)

user_input = [
                "MAC에 대해서 자세히 알려주세요.",
                "컴파일러가 무엇인가요??", 
                "TCP와 UDP의 차이점은 무엇인가요?", 
                "데이터베이스에 대해 알려주세요.",
                "TCP의 이점은 무엇인가요?"
            ]

for i in user_input:
    main(i)
    print("\n-----------------------------------\n")
    


🧠 Embedding the input question...
🔍 Comparing with existing documents...
Assistant: {'LLMAnswer': None, 'userQuestion': 'MAC에 대해서 자세히 알려주세요.'}

-----------------------------------

🧠 Embedding the input question...
🔍 Comparing with existing documents...
Assistant: {'LLMAnswer': None, 'userQuestion': '컴파일러가 무엇인가요??'}

-----------------------------------

🧠 Embedding the input question...
🔍 Comparing with existing documents...

가장 유사한 질문: '5
2024-08-20
스위치와 허브의 차이점은?
스위치는 데이터 패킷을 특정 장치로 전달하는 반면, 허브는 모든 장치로 데이터를 전송합니다.
6
2024-08-20
IP 주소란?
IP 주소는 네트워크 상에서 장치들을 식별하기 위한 고유 주소입니다.
7
2024-08-20
DNS의 역할은?
DNS는 도메인 이름을 IP 주소로 변환하는 시스템입니다.
8
2024-08-20
방화벽이란?
방화벽은 네트워크 보안을 위해 외부의 불법적인 접근을 차단하는 시스템입니다.
9
2024-08-20
VPN의 기능은?
VPN은 공용 네트워크에서 사설 네트워크를 사용하는 것처럼 보안을 제공하는 기술입니다.
10
2024-08-20
패킷이란?'
유사도: 85.97%
Assistant: {'LLMAnswer': 'TCP(Transmission Control Protocol)와 UDP(User Datagram Protocol)는 둘 다 인터넷 프로토콜 스위트의 일부로, 데이터 전송을 위한 프로토콜입니다. 그러나 이 두 프로토콜은 데이터 전송 방식에서 여러 가지 중요한 차이점이 있습니다.\n\n1. **연결 지향